# Notebook for visualizing the disk scheme on map

In [1]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


In [2]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_disk import DiskLSH
from constants import  *
from utils.helpers import file_handler


## Define Disk

In [3]:
DiskLSH = DiskLSH(
        name="Disk1",
        min_lat = R_MIN_LAT,
        max_lat=R_MAX_LAT,
        min_lon=R_MIN_LON,
        max_lon=R_MAX_LON,
        disks=50,
        layers=3,
        diameter=0.5,
        meta_file="meta.txt",
        data_path="data",
)


## Visualize disk on map

In [4]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_disks_with_boundary(disk_lsh):
    """
    Visualizes the disks of the DiskLSH object using Folium and adds a bounding box.

    Parameters:
    - disk_lsh (DiskLSH): An instance of the DiskLSH class.

    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (disk_lsh.min_lat + disk_lsh.max_lat) / 2
    center_lon = (disk_lsh.min_lon + disk_lsh.max_lon) / 2

    # Initialize folium map
    map_disks = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_disks)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(disk_lsh.min_lat, disk_lsh.min_lon), (disk_lsh.max_lat, disk_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the disk structure
    for layer_index, (layer, disks) in enumerate(disk_lsh.disks.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_disks.add_child(layer_group)

        # Plot disks as circles
        for disk in disks:
            lat, lon = disk  # Disk center
            folium.Circle(
                location=[lat, lon],
                radius=disk_lsh.diameter * 500,  # Convert km to meters
                color=color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1}\nDisk: ({lat:.5f}, {lon:.5f})",
            ).add_to(layer_group)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_disks)

    return map_disks

# Example usage
disk_map = visualize_disks_with_boundary(DiskLSH)
disk_map.save("disk_visualization.html")


## Add trajectory


In [5]:
trajs_1 = file_handler.load_all_trajectory_files(ROME_OUTPUT_FOLDER_FULL, "R")

trajs_2 = file_handler.load_trajectory_files(["R_ABA.txt"], ROME_OUTPUT_FOLDER_FULL) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values
print(trajs_2)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/thomasnitsche/Developer/skole/master/code/masteroppgave/dataset/rome/output/'

### Hash trajectory

In [426]:
hash_num = DiskLSH._create_trajectory_hash_with_KD_tree_numerical(trajs_2["R_ABA"])
hash_letter = DiskLSH._create_trajectory_hash(trajs_2["R_ABA"])


print(hash_num)
print(hash_letter)

(41.90114, 12.49354)

[[array([41.91805681, 12.48990845]), array([41.91886387, 12.4909163 ]), array([41.91886387, 12.4909163 ]), array([41.91805681, 12.48990845])], [array([41.91780273, 12.49252981]), array([41.91780273, 12.49252981]), array([41.91780273, 12.49252981]), array([41.91780273, 12.49252981]), array([41.90504959, 12.48958955]), array([41.90642968, 12.49209156]), array([41.9007033 , 12.49032573])], [array([41.91991165, 12.49534651]), array([41.91924074, 12.49645619])]]
[['AW', 'BM', 'AW', 'BM'], ['AQ', 'AQ', 'AQ', 'AQ', 'AL', 'AO', 'BX'], ['BX', 'BE']]


(41.90114, 12.49354)

## Visualize disk scheme, trajectory and hashed trajectory on map

In [427]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random

def visualize_disks_with_trajectory(disk_lsh, trajectory, hashed_trajectory):
    """
    Visualizes the disks of the DiskLSH object using Folium.
    - Trajectory is displayed as a blue polyline.
    - Hashed points are marked with colored circles.

    Parameters:
    - disk_lsh (DiskLSH): An instance of the DiskLSH class.
    - trajectory (list): List of (lat, lon) coordinates representing a single trajectory.
    - hashed_trajectory (list of lists): Hashed trajectory representation per layer.

    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (disk_lsh.min_lat + disk_lsh.max_lat) / 2
    center_lon = (disk_lsh.min_lon + disk_lsh.max_lon) / 2

    # Initialize folium map
    map_disks = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different disk layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_disks)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(disk_lsh.min_lat, disk_lsh.min_lon), (disk_lsh.max_lat, disk_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the disk structure
    for layer_index, (layer, disks) in enumerate(disk_lsh.disks.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_disks.add_child(layer_group)

        # Plot disks as circles
        for disk in disks:
            lat, lon = disk  # Disk center
            folium.Circle(
                location=[lat, lon],
                radius=disk_lsh.diameter * 500,  # Convert km to meters
                color=color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1}\nDisk: ({lat:.5f}, {lon:.5f})",
            ).add_to(layer_group)

    # Plot the trajectory as a blue polyline
    folium.PolyLine(
        trajectory,
        color="blue",
        weight=2.5,
        opacity=1,
        popup="Original Trajectory"
    ).add_to(base_layer)

    # Add markers for each point along the trajectory
    for lat, lon in trajectory:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Small marker
            color="black",
            fill=True,
            fill_opacity=1,
            popup=f"Point: ({lat:.5f}, {lon:.5f})"
        ).add_to(base_layer)

    # Plot hashed trajectory points as markers with circles
    hashed_layer_colors = ["red", "blue", "green", "purple", "orange"]  # Colors for different layers

    for layer_index, layer_points in enumerate(hashed_trajectory):
        layer_color = hashed_layer_colors[layer_index % len(hashed_layer_colors)]  # Assign color per layer

        for lat, lon in layer_points:
            # Add a small circle around the hashed point
            folium.Circle(
                location=(lat, lon),
                radius=20,  # Small circle
                color=layer_color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1} Hashed Point"
            ).add_to(base_layer)

            # Add a marker on top of the hashed point
            folium.Marker(
                location=(lat, lon),
                icon=folium.Icon(color=layer_color, icon="info-sign"),
                popup=f"Hashed Point ({lat:.5f}, {lon:.5f}) - Layer {layer_index + 1}"
            ).add_to(base_layer)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_disks)

    return map_disks


disk_map = visualize_disks_with_trajectory(DiskLSH, trajs_2["R_ABA"], hash_num)
disk_map.save("disk_with_hashed_trajectory.html")
